### Ideal RSA Model -- Fully Observed Environment and Utterance Space

**What’s in this notebook?** This notebook describes the listener model in the open world Boxworld domain, encoding what pragmatic listener might reasonably have in mind when trying to interpret a given utterance.
This notebook is a prototype that can ultimately be abstracted away for later use.

In [703]:
using Gen
using JSON
using Distances

include("dirichlet.jl")

**1: A generative model of a stockroom.** 

We'll begin by defining some primitives for the Stockroom environment. These model the same primitives in the boxworld/world.py library used to generate the Stockroom dataset:
a `Box`, an `Aisle` that contains a set of boxes, and a `World` containing a set of aisles.

We'll also define some utilities for serializing and deserializing these data structures.

In [704]:
module Stockroom
MIN_BOX_SIZE = 0.1
MIN_BOX_SPACING = 0.1

    function clamp_min_max(x, min, max)
        if x <= min
            return min
        elseif x >= max
            return max
        else 
            return x
        end
     end 

    mutable struct Box
        id::Int
        pos::Tuple{Int, Float64}  # Pos is in (shelf_id, x_distance on shelf)
        width::Float64
        height::Float64
        aisle_id::Int
    end 
    function init_clamped_box(id, pos, width, height, aisle)
        shelf_height = aisle.size / aisle.num_shelves
        width = clamp_min_max(width, MIN_BOX_SIZE, aisle.size - MIN_BOX_SIZE)
        height = clamp_min_max(height, MIN_BOX_SIZE, shelf_height - MIN_BOX_SIZE)
        return Box(id, pos, width, height, aisle.id)
    end
    function deserialize_box(box_dict)
        return Box(box_dict["id"] + 1,
                  (box_dict["pos"][1], box_dict["pos"][2]),
                   box_dict["width"],
                   box_dict["height"],
                   box_dict["aisle_id"] + 1)
    end 

    mutable struct Aisle
        id::Int
        size::Float64
        num_shelves::Float64
        boxes::Vector{Box}
        function Aisle(id, size, num_shelves, boxes)
            shelf_height = size / num_shelves
            function clamp_boxes!(shelf_height::Float64, boxes::Vector{Box})
                for box in boxes
                    box.width = clamp_min_max(box.width, MIN_BOX_SIZE, size - MIN_BOX_SIZE)
                    box.height = clamp_min_max(box.height, MIN_BOX_SIZE, shelf_height - MIN_BOX_SIZE)
                    box.aisle_id = id
                end 
                return boxes
            end 
            new(id, size, num_shelves, clamp_boxes!(shelf_height, boxes))
        end 
    end 
    Aisle(id, size, num_shelves) = Aisle(id, size, num_shelves, Box[])
    function room_left(aisle::Aisle, box::Box, distance::Float64)
        # Returns true if it is possible to place the box on the current aisle.
        # Assumes boxes are sorted.
        if length(aisle.boxes) < 1
            last_box_shelf = 1
            last_box_start = 0
            last_box_width = 0
        else
            last_box = last(aisle.boxes)
            last_box_shelf = last_box.pos[1]
            last_box_start = last_box.pos[2]
            last_box_width = last_box.width
        end
        if (last_box_shelf < aisle.num_shelves) && box.width <= aisle.size
            return true # At least one shelf left that can fit the box.
        else
            new_end = last_box_start + last_box_width + distance + box.width
            return new_end <= aisle.size # Last shelf, and box doesn't spill over
        end
    end

    function place_box!(aisle::Aisle, box::Box, distance::Float64)
        # Places a box on the aisle, wrapping the remaining distance onto the next shelf
        # if there is no room left. Assumes that there is room left in the aisle as a whole.
        if length(aisle.boxes) < 1
            last_box_shelf = 1
            last_box_start = 0
            last_box_width = 0
        else
            last_box = last(aisle.boxes)
            last_box_shelf = last_box.pos[1]
            last_box_start = last_box.pos[2]
            last_box_width = last_box.width
        end
        new_shelf = last_box_shelf
        new_start = last_box_start + last_box_width + distance
        new_end = new_start + box.width
        if new_end >= aisle.size
            new_shelf = new_shelf + 1
            if new_start <= aisle.size
                new_start = 0
            else
                new_start = new_start - aisle.size
            end
        end
        box.pos = (new_shelf, new_start)
        box.aisle_id = aisle.id
        push!(aisle.boxes, box)
    end 
    

    function deserialize_aisle(aisle_dict)
        id = aisle_dict["id"] + 1
        size = aisle_dict["size"]
        num_shelves = aisle_dict["num_shelves"]
        boxes = [deserialize_box(b) for b in aisle_dict["boxes"]]
        return Aisle(id, size, num_shelves, boxes)
    end 

    struct World
        aisles::Vector{Aisle}
    end 
    function deserialize_world(world_dict)
        return World([deserialize_aisle(a) for a in world_dict["aisles"]])
    end 
    struct Situation
        world::World
        location::Int
        instruction::String
        target::Int
        meaning
    end 
    function deserialize_situation(situation_dict)
        world = deserialize_world(situation_dict["world"])
        location = situation_dict["location"]
        instruction = situation_dict["instruction"]
        target = situation_dict["target"]
        meaning = situation_dict["meaning"]
        return Situation(world, location, instruction, target, meaning)
    end 

end

function load_situation(situation_file)
    curr_dir = dirname(@__DIR__)
    dataset_dir = "boxworld/data/demo_dataset"
    full_path = join([curr_dir, dataset_dir, situation_file], "/")
    open(full_path, "r") do f
        return Main.Stockroom.deserialize_situation(JSON.parse(f))
    end
end 
function serialize_world_as_situation(world, situation_file)
    
end 

serialize_world_as_situation (generic function with 1 method)

Next, we'll actually define a generative model for the stockroom itself. A stockroom is a set of aisles, populated by boxes. One reasonable assumption we could make is that stockrooms there are actually 1 or more categories of boxes, and stockrooms are 
populated by placing boxes onto shelves as they are generated until no more fit.

We'll define this model below, and some helper utility distributions.

In [705]:
PARAMS = (
    num_aisles = 5,
    distance_scale = 1,
    box_scale = 0.5
)

# A distribution that is guaranteed to be 1 or higher.
@dist poisson_plus_one(rate) = poisson(rate) + 1;


@gen function stockroom_model(aisle_size::Float64, num_shelves::Float64, params)
    # Generate a categorical distribution over 1 or more box_types, each of which defines
    # it's own distribution over box heights and widths.
    num_box_types ~ poisson_plus_one(1) # Generate number of box types -- at least 1
    box_fractions ~ dirichlet([1.0 for i=1:num_box_types])
    
    shelf_height = aisle_size / num_shelves
    box_types = [
        (
         {(:box_mean_width, i)} ~ gamma(shelf_height * 0.5, shelf_height * 0.25),
         {(:box_mean_height, i)} ~ gamma(shelf_height * 0.5, shelf_height * 0.25),
        )
         for i = 1:num_box_types
    ]
    @dist choose_box_type() = box_types[categorical(box_fractions)]
    
    num_aisles ~ gamma(params.num_aisles, 1)
    distance_shape ~ gamma(aisle_size / num_shelves, 1)
    distance_scale ~ gamma(params.distance_scale, 1)
    
    # For now, we;ll simply start at the start of each aisle and place boxes until no more fit.
    aisles = [Main.Stockroom.Aisle(id, aisle_size, num_shelves) for id=0:num_aisles] # N.B. 1-indexing
    curr_aisle = 1
    box_id = 1
    while curr_aisle <= length(aisles)
        box_type = {(:box_type, box_id)} ~ choose_box_type()
        box_width = {(:box_width, box_id)} ~ gamma(box_type[1], params.box_scale)
        box_height = {(:box_height, box_id)} ~ gamma(box_type[2], params.box_scale)
        box = Main.Stockroom.init_clamped_box(box_id, (0,0), box_width, box_height, aisles[curr_aisle])
        distance_from_last_box = {(:distance_from_last_box, box_id)} ~ gamma(distance_shape, distance_scale)
        if Main.Stockroom.room_left(aisles[curr_aisle], box, distance_from_last_box)
            Main.Stockroom.place_box!(aisles[curr_aisle], box, distance_from_last_box)
        else 
            if curr_aisle == length(aisles)
                break
            else
                curr_aisle += 1
                Main.Stockroom.place_box!(aisles[curr_aisle], box, distance_from_last_box)
            end 
        end 
        box_id += 1
    end
    return aisles
end 

DynamicDSLFunction{Any}(Dict{Symbol,Any}(), Dict{Symbol,Any}(), Type[Float64, Float64, Any], false, Union{Nothing, Some{Any}}[nothing, nothing, nothing], ##stockroom_model#482, Bool[0, 0, 0], false)

We'll define some visualization code to explore what the sampled stockrooms look like. This uses the viewer in boxworld/viewer/viewing_interface.html.

In [706]:
aisle_size = 12
num_shelves = 4
tr = simulate(stockroom_model, (aisle_size, num_shelves, PARAMS))
get_choices(tr)

│
├── (:box_height, 361) : 2.399991615876884
│
├── (:box_height, 196) : 1.166016720208298
│
├── (:box_height, 354) : 0.2243389023400538
│
├── (:distance_from_last_box, 321) : 0.3145332055870854
│
├── (:box_height, 28) : 0.41671612128684704
│
├── (:box_width, 397) : 0.585781828105547
│
├── (:box_height, 398) : 0.9616736142449606
│
├── (:box_height, 47) : 0.1622616440098034
│
├── (:box_width, 76) : 1.317468380634899
│
├── (:box_type, 162) : (1.5090464346837122, 1.0268272661742641)
│
├── (:distance_from_last_box, 202) : 0.41237722575579655
│
├── (:box_type, 370) : (1.5090464346837122, 1.0268272661742641)
│
├── (:box_width, 383) : 1.9219514990189095
│
├── (:box_width, 393) : 2.2360544857971627
│
├── (:box_width, 444) : 0.1604020484537735
│
├── (:box_width, 251) : 0.1280236909938419
│
├── (:box_height, 95) : 0.04491564358242395
│
├── (:box_height, 402) : 0.050905846609854456
│
├── (:distance_from_last_box, 417) : 0.30767848530765596
│
├── (:box_height, 34) : 0.8283985361272526
│
├── (:box_w

In [ ]:
# TODO: check generative model.
# possibly too many boxes on one shelf
# TBD: deserialize and visualize; put the visualization commands here.
# and maybe set an endpoint so we can just refresh the visualizatoin page.


**2: A simple (vague) referential semantics.** 

Next, we'll define semantic primitives and a linguistic grammar for this environment.

The `Sentence` data structure contains both a `String` and a `Meaning`. A `Meaning` is a program that can be run on a Stockroom `World`. Currently, we'll model meanings as a set of declarative statements about the world; if there are no free variables, we should be able to evaluate the program (and return a satisfying assignment).

We'll also define some utilities for executing and checking `Meaning`s on various `World`s.

In [818]:
# Utility functions
MAX_DISTANCE = 100
function nearest_distance(b1::Main.Stockroom.Box, b2::Main.Stockroom.Box, world::Main.Stockroom.World)
    # Returns approximate distance between nearest points between two boxes
    if b1.aisle_id != b2.aisle_id
        return MAX_DISTANCE
    end
    shelf_size = world.aisles[1].size
    num_shelves = world.aisles[1].num_shelves
    shelf_height = shelf_size / num_shelves
    
    if b1.pos[1] == b2.pos[1]
        if b1.pos[2] < b2.pos[2]
            return b2.pos[2] - (b1.pos[1] + b1.width)
        else
            return b1.pos[2] - (b2.pos[1] + b2.width)
        end 
    elseif b1.pos[1] < b2.pos[1]
        top_box = b1
        bottom_box = b2
    else
        top_box = b2
        bottom_box = b1
    end 
        top_y = top_box.pos[1] * shelf_height
        bottom_y = (bottom_box.pos[1] * shelf_height) - bottom_box.height
        top_points = [[top_box.pos[2], top_y], 
                      [top_box.pos[2] + (0.5 * top_box.width), top_y], 
                      [top_box.pos[2] + top_box.width, top_y]]
        bottom_points = [[bottom_box.pos[2], bottom_y], 
                      [bottom_box.pos[2] + (0.5 * bottom_box.width), bottom_y], 
                      [bottom_box.pos[2] + bottom_box.width, bottom_y]]
        dists = vec([euclidean(x,y) for x in top_points, y in bottom_points])
        return min(dists...)
end


# Predicates
is_plural(x::Set{Any}) = length(x) > 1
is_singleton(x::Set{Any}) = length(x) == 1
is_tall(x::Main.Stockroom.Box, tall_threshold) = x.height > tall_threshold
is_tall(x::Set{Main.Stockroom.Box}, tall_threshold) = all([is_tall(elem, tall_threshold) for elem in x])
is_short(x::Main.Stockroom.Box, short_threshold) = x.height < short_threshold
is_short(x::Set{Main.Stockroom.Box}, short_threshold) = all([is_short(elem, short_threshold) for elem in x])

is_near(x::Main.Stockroom.Box, 
        y::Main.Stockroom.Box; 
        near_threshold, world::Main.Stockroom.World) = x != y && nearest_distance(x, y, world) < near_threshold
is_near(x::Vector{Main.Stockroom.Box}, y::Vector{Main.Stockroom.Box};
        near_threshold, world::Vector{Main.Stockroom.Box}) = 
        any([is_near(x, y, near_threshold, world)])

# Quantifiers
# N.B. double quoting not necessary -- pass filters as filter? since should already
# use namespace?
# function exists(x, filters; world) 
#     # A version of 'exists' that only checks whether a set of boxes exists in the same 
#     # aisle as x that satisfies all of the filtres
    
#     candidate_sets = # Other candidate boxes in the same aisle.
#     remaining = candidate_sets
#     for f in filters
#         remaining = [f(y, world, kwargs) for y in remaining]
#     return len(remaining) > 1
# end 
    


is_near (generic function with 5 methods)

In [826]:
module Language
    mutable struct Var
        name::String
        t::Type
        val::Any
    end
    function is_free(v::Var)
        return v.val == nothing 
    end
    function build_assignment_expr(v::Var)
        return Expr(Symbol("="), Symbol(v.name), v.val)
    end 
    
    mutable struct Meaning
        predicates::Vector{Any}
        free_vars::Vector{Var}
    end

    mutable struct Sentence
        string::String
        meaning::Meaning
        depth::Int
    end
    
    function all_free_variables(sentences::Vector{Sentence}, assignments::Dict{String, Var})
        # Returns unassigned free variables in the sentences
        free_variables = Dict{String, Var}()
        for s in sentences
            for v in s.meaning.free_vars
                if !(haskey(assignments, v.name))
                    free_variables[v.name] = v
                end
            end
        end
        return free_variables
    end

    abstract type Grammar end
    
    mutable struct HardCodedGrammar <: Grammar
        sentences::Vector{Sentence}
        valuation_fn::Function
    end

    function possible_sentences(grammar::Grammar; max_depth, enumerate_all)
        if isa(grammar, HardCodedGrammar)
            return [s for s in grammar.sentences if s.depth <= max_depth]
        else
            throw(ErrorException("TODO: implement grammar expansion."))
        end
    end 
end 


function possible_goal_interpretations(world::Main.Stockroom.World, 
                                       goal_assignments::Dict{String, Main.Language.Var})
    # Given a world, returns a set of satisfying assignments for each variable
    all_variable_assignments = Dict{String, Set{Main.Language.Var}}()
    for goal_assign in goal_assignments
        v = goal_assign.second
        if v.t == Main.Stockroom.Box
            variable_assignments = Set([
                    Main.Language.Var(v.name, v.t, Set([box]))
                    for box in Iterators.flatten((a.boxes for a in world.aisles))
                    ])
            all_variable_assignments[v.name] = variable_assignments
        else
            throw(ErrorException("Not supported for type" * string(v.t)))
        end
    end 
    return all_variable_assignments
end 

function query_same_assignment(query::Dict{String, Main.Language.Var}, assignments::Dict{String, Main.Language.Var})
    # Returns true if all of the variables in the query are assigned to the same values
    # in the target.
    for v in values(query)
        if assignments[v.name].val != v.val
            return false
        end
    end 
    return true
end

function evaluate(world, meaning::Main.Language.Meaning, assignments::Dict{String, Main.Language.Var})
    # For now, implicitly assumes 'meanings' to be a set of truth-conditional declarative
    # predicates. Returns true if all of these evaluate to true given assignments
    # and the world.
    for var in values(assignments)
        eval(Main.Language.build_assignment_expr(var))
    end 
    return all([eval(fact) for fact in meaning.predicates])
end 

evaluate (generic function with 26 methods)

For demonstration purposes, we'll hardcode an extremely simple `Grammar` that we can use to test our model.

In [827]:
a_box_predicate = :(is_singleton(target_box))
a_tall_box_predicate = :(is_tall(target_box, tall_threshold))
a_short_box_predicate = :(is_short(target_box, short_threshold))
target_box_var = Main.Language.Var("target_box", Set{Main.Stockroom.Box}, nothing)
tall_threshold_var = Main.Language.Var("tall_threshold", Float64, nothing)
short_threshold_var = Main.Language.Var("short_threshold", Float64, nothing)

a_box_meaning = Main.Language.Meaning([a_box_predicate], [target_box_var])
a_tall_box_meaning = Main.Language.Meaning([a_box_predicate, a_tall_box_predicate], [target_box_var, tall_threshold_var])
a_short_box_meaning = Main.Language.Meaning([a_box_predicate, a_short_box_predicate], [target_box_var, short_threshold_var])

s1 = Main.Language.Sentence("it's a box", a_box_meaning, 1)
s2 = Main.Language.Sentence("it's a tall box", a_tall_box_meaning, 1)
s3 = Main.Language.Sentence("it's a short box", a_short_box_meaning, 1)

function toy_valuations(variable, world)
    # Defines the possible valuations for a given variable. 
    if variable.t == Float64
        aisle_size = world.aisles[1].size
        return ("continuous", 0, aisle_size * aisle_size)
    else
        throw(ErrorException("Not supported for type" * string(v.t)))
    end
end 

toy_grammar = Main.Language.HardCodedGrammar([s1, s2, s3], toy_valuations)

Main.Language.all_free_variables([s3], Dict{String, Main.Language.Var}())



Dict{String,Main.Language.Var} with 2 entries:
  "target_box"      => Var("target_box", Set{Box}, nothing)
  "short_threshold" => Var("short_threshold", Float64, nothing)

We'll load the `small box near big boxes` example world as our running example.

In [828]:
situation_file = "demo_a_small_box_near_big_boxes_0_0.json"
world = load_situation(situation_file).world;

In [829]:
# Demo: possible goal interpretations
v = Main.Language.Var("target_box", Main.Stockroom.Box, nothing)
Main.Language.is_free(v)
goal_assignment = Dict(v.name => v)
possible_goals = possible_goal_interpretations(world, goal_assignment);


**3: Ideal RSA Speaker and Listener: fully observed world; complete synthetic utterance set.** 

Next, we'll define a model for an *ideal* pragmatic speaker and listener pair that often appears in the literature -- one in which the pair reasons about a complete space of possible alternative sentences with a predefined semantics, and both can see the entire world the speaker could be referring to.

In this model, the *literal* listener receives a full valuation V for any free variables.

In [830]:
function literal_satisfying_interpretations(sentence::Main.Language.Sentence,
                                            world::Main.Stockroom.World,
                                            assigned_variables::Dict{String, Main.Language.Var},
                                            possible_goal_interpretations::Dict{String, Set{Main.Language.Var}})
    # Returns the subset of assignments in possible_goal_interpretations where the sentence
    # does not evaluate to false.
    m = sentence.meaning
    satisfying_interpretations = []
    for goal_interpretation in Iterators.product(values(possible_goal_interpretations)...)
        # Add all of the assignments to the assigned_variables
        goal_assignments = Dict(v.name => v for v in goal_interpretation)
        full_assignment = merge(assigned_variables, goal_assignments)
        valuation = evaluate(world, m, full_assignment)
        if valuation
            push!(satisfying_interpretations, goal_assignments)
        end 
    end 
    return satisfying_interpretations
end 

function ideal_literal_listener_likelihoods(;sentence::Main.Language.Sentence, 
        world::Main.Stockroom.World, assigned_variables::Dict{String, Main.Language.Var}, goal_assignments::Dict{String, Main.Language.Var})
    # Calculates the (analytical) likelihood that a listener will assign the 
    # goal variables in a sentence as desired, given the state of the world and 
    # some assigned variables
    
    # Calculate a subset of ‘cases’ to consider given the world and goal assignments -- e.g. 
	# all possible target boxes.
    possible_goals = possible_goal_interpretations(world, goal_assignments)
    
    # Calculate which of the possible actions literally satisfies the sentence in the world.
    literal_satisfying = literal_satisfying_interpretations(sentence, world, assigned_variables,
                                                            possible_goals)
    # Calculate which of the possible actions actually leads to the goal.
    if length(literal_satisfying) < 1
        return 0
    end 
    correct_goal = [i for i in literal_satisfying if query_same_assignment(goal_assignments, i)]
    return length(correct_goal) / length(literal_satisfying)
end 

# Demo with hard-coded assigned variables.
goal  = Main.Language.Var("target_box", Main.Stockroom.Box, Set([world.aisles[1].boxes[1]]))
goal_assignments = Dict(goal.name => goal)
println(s1)
println(goal_assignments)
ideal_literal_listener_likelihoods(sentence=s1, world=world, assigned_variables=Dict{String, Main.Language.Var}(),
                                  goal_assignments=goal_assignments)

Main.Language.Sentence("it's a box", Main.Language.Meaning(Any[:(is_singleton(target_box))], Main.Language.Var[Main.Language.Var("target_box", Set{Main.Stockroom.Box}, nothing)]), 1)
Dict("target_box" => Main.Language.Var("target_box", Main.Stockroom.Box, Set(Main.Stockroom.Box[Main.Stockroom.Box(1, (0, 7.0), 2.6378991413640844, 3.231005277954553, 1)])))


0.027777777777777776

In [871]:
function per_word_cost_function(sentence::Main.Language.Sentence, speaker_params)
    n_words = length(split(sentence.string))
    return n_words * speaker_params.COST_PER_WORD
end

@gen function ideal_rsa_speaker_model(world::Main.Stockroom.World, 
        grammar::Main.Language.Grammar, speaker_params)
    
    world_boxes = collect(Iterators.flatten((a.boxes for a in world.aisles)))
    @dist choose_target_box() = world_boxes[uniform_discrete(1, length(world_boxes))]
    target_box ~ choose_target_box()
    
    goal_assignment = Dict("target_box" => Main.Language.Var("target_box", Main.Stockroom.Box, Set([target_box])))
    assignments = Dict(
        "target_box" => Main.Language.Var("target_box", Main.Stockroom.Box, Set([target_box]))
    )
    
    # To decide what to say, the speaker first chooses a valuation for all of the possible
    # free variables in their vocabulary.
    possible_sentences = collect(Main.Language.possible_sentences(grammar, max_depth=1, enumerate_all=true))
    free_variables = Main.Language.all_free_variables(possible_sentences, assignments)
    
    # For now, we'll assume a very simple uniform prior over possible valuations.
    for free_variable in values(free_variables)
        possible_values = grammar.valuation_fn(free_variable, world)
        if possible_values[1] == "continuous"
            lower = possible_values[2]
            upper = possible_values[3]
            free_variable_value = {(:speaker_valuation, free_variable.name)} ~ uniform(lower, upper)
        elseif possible_values[1] == "discrete"
            choices = possible_values[2]
            @dist choose_valuation() = choices[uniform_discrete(1, length(choices))]
            free_variable_value = {(:speaker_valuation, free_variable.name)} ~ choose_valuation()
        end
        free_variable.val = free_variable_value
        assignments[free_variable.name] = free_variable
    end
    
    # Now, the speaker evaluates the likelihood that a listener with those valuations will give
    # them their intended box.
    # They weight each sentence proportionate to the likelihood of a listener choosing the goal, with some cost
    sentence_weights = Dict()
    for s in possible_sentences
        literal_likelihood = ideal_literal_listener_likelihoods(sentence=s, world=world, 
                                                                assigned_variables=assignments, goal_assignments=goal_assignment)
        sentence_weight = exp(speaker_params.SPEAKER_OPTIMALITY * (log(literal_likelihood) - speaker_params.SENTENCE_COST_FN(s, speaker_params)))
        sentence_weights[s] = sentence_weight
    end 
    # The speaker now chooses a sentence proportional to that weighted distribution.
    normalized_sentence_weights = [sentence_weights[s] for s in possible_sentences]
    normalized_sentence_weights = normalized_sentence_weights ./ sum(normalized_sentence_weights)
    
    possible_sentence_strings = [s.string for s in possible_sentences]
    @dist choose_sentence_string() = possible_sentence_strings[categorical(normalized_sentence_weights)]
    speaker_sentence_string ~ choose_sentence_string()
    return speaker_sentence_string, sentence_weights
end 

SPEAKER_PARAMS = (
    SPEAKER_OPTIMALITY = 1.0,
    COST_PER_WORD = 0.1,
    SENTENCE_COST_FN = per_word_cost_function
)


tr = simulate(ideal_rsa_speaker_model, (world, toy_grammar, SPEAKER_PARAMS))
get_choices(tr)

│
├── (:speaker_valuation, "short_threshold") : 230.52463573434437
│
├── :speaker_sentence_string : it's a box
│
├── :target_box : Main.Stockroom.Box(30, (3, 7.337844702065779), 1.3562822668898953, 1.8147236077046502, 5)
│
└── (:speaker_valuation, "tall_threshold") : 36.24411219844529


We'll define some visualization code to explore the speaker's choices.

Finally, we'll define the ideal RSA listener -- for now, our listener will perform naive importance sampling inference, though we can (and will almost definitely need to) adjust this to scale further.

In [882]:
function ideal_rsa_listener_model(;sentence::Main.Language.Sentence, 
        world::Main.Stockroom.World, 
        grammar::Main.Language.Grammar,
        speaker_params,
        num_samples)
    # The listener now attempts to infer the meaning of all of the remaining variables in the 
    # sentence, given the model of the speaker, and conditioned on 
    observations = Gen.choicemap()
    observations[:speaker_sentence_string] = sentence.string
    
    (trace, _) = importance_resampling(ideal_rsa_speaker_model, (world, grammar, speaker_params), observations, num_samples)
    return trace
end 

possible_sentences = collect(Main.Language.possible_sentences(toy_grammar, max_depth=1, enumerate_all=true))

demo_sentence = possible_sentences[2]
println("Observed sentence: " * demo_sentence.string)
tr = ideal_rsa_listener_model(sentence=demo_sentence, world=world, grammar=toy_grammar, speaker_params=SPEAKER_PARAMS, num_samples=100)

get_choices(tr)

Observed sentence: it's a tall box


│
├── (:speaker_valuation, "short_threshold") : 203.8382547022773
│
├── :speaker_sentence_string : it's a tall box
│
├── :target_box : Main.Stockroom.Box(35, (3, 7.0), 2.0305291065247113, 3.245981703982462, 6)
│
└── (:speaker_valuation, "tall_threshold") : 2.8286975553295974


In [839]:
# TODO: define some visualization code to see the inferred thresholds and the inferred targets / distribution.
# This should replicate the Lassiter and Goodman findings.

# Also check that this changes depending on the box sizes 
# Do we reasonably pick a tall box?
# Run this N times and see what we pick

importance_resampling (generic function with 2 methods)